# Social graph and Interactions

# Assignment 2

**Group 30:**

Aleksandar Lukic, s194066

Clara Mejlhede Lorenzen, s180350

Julia Katarzyna Zalewska, s244491

## Python libraries used in this assignment

In [1]:
import json
import math
import matplotlib as mpl
from matplotlib import colormaps
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import os
import random
import re
import urllib.request

## Part 1: Genres and communities and plotting

#### <font color='gray'>  Write about genres and modularity. </font>

#### <font color='gray'> Detect the communities, discuss the value of modularity in comparison to the genres.</font>

#### <font color='gray'> Calculate the matrix $D$ and discuss your findings.</font>

#### <font color='gray'> Plot the communities and comment on your results.</font>

## Part 2: TF-IDF to understand genres and communities

#### <font color='gray'> Explain the concept of TF-IDF in your own words and how it can help you understand the genres and communities. </font>

#### <font color='gray'> Calculate and visualize TF-IDF for the genres and communities.</font>

#### <font color='gray'> Use the matrix D (Lecture 7, part 2) to dicusss the difference between the word-clouds between genres and communities.</font>

## Part 3: Sentiment of the artists and communities

#### <font color='gray'> Calculate the sentiment of the Artists pages (OK to work with the sub-network of artists-with-genre) and describe your findings using stats and visualization, inspired by the first exercise of week 8.</font>

#### <font color='gray'> Discuss the sentiment of the largest communities. Do the findings using TF-IDF during Lecture 7 help you understand your results?</font>